In [1]:
from glob import glob
from tqdm import tqdm
from unidecode import unidecode
from sklearn.feature_extraction.text import CountVectorizer
import json
import os
import mp
import re
import string

timestamps = [i * 0.02 for i in range(1500 + 1)]

rejected_c = set(string.digits) | set(string.punctuation)
def clean(s):
    for r in rejected_c:
        s = s.replace(r, ' ')
    return re.sub(r'[ ]+', ' ', ''.join(s)).strip()

In [2]:
from huggingface_hub import snapshot_download
snapshot_download(
    repo_id="mesolitica/malay-dialect-dictionary-instructions", 
    repo_type = 'dataset',
    local_dir = './malay-dialect-dictionary-instructions')

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

'/home/husein/ssd3/malay-dialect-dictionary-instructions'

In [3]:
keywords = set()

for f in tqdm(glob('malay-dialect-dictionary-instructions/*jsonl')):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            k = clean(unidecode(l['left'].lower())).split()
            for k_ in k:
                keywords.add(k_)
len(keywords)

100%|██████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 113.80it/s]


5979

In [4]:
files = glob('dialects_processed/**/*/*.json', recursive = True)

len(files)

147181

In [5]:
rejected = [
    'terima kasih kerana menonton',
    'terima kasih',
    'thank you for watching',
]

In [6]:
def new_path(f):
    f = f.replace('.mp3', '.alignment')
    f = f.replace('_processed/', '_processed_alignment/')
    return f

def loop(files):
    files, _ = files
    data = []
    for file in tqdm(files):
        folder = os.path.split(file)[0]
        folder_folder = os.path.split(folder)[1]
        filename = file.replace('.json', '')

        try:
            with open(file) as fopen:
                d = json.load(fopen)
        except:
            continue

        for no, obj in enumerate(d):
            text = obj["text"].strip()
            
            rt_ = re.sub('[^a-z ]+', '', text.lower()).strip()
            if any([s == rt_ for s in rejected]):
                continue
            
            if not len(set(clean(text.lower()).split()) & keywords):
                continue
            
            try:
                dense = CountVectorizer(ngram_range = (3,3)).fit_transform([text]).todense()
                repeat = (dense > 3).sum() >= 1
                if repeat:
                    continue
            except:
                continue
            
            audio_path = os.path.join(folder, f'{folder_folder}_{no}.mp3')
            align_path = new_path(audio_path)
            
            if not os.path.exists(align_path):
                continue
                
            with open(align_path) as fopen:
                align = json.load(fopen)
                
            scores = [a for a in align if a['score'] <= -15]
            if len(scores):
                continue

            data.append({
                'audio': audio_path,
                'alignment': align_path,
            })
    
    return data

In [7]:
d = loop((files[-10:], 0))

100%|██████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 612.91it/s]


In [8]:
d[0]

{'audio': 'dialects_processed/‘Zaman PN harga barang naik mencanak!’ - Rafizi [OjJ6hCKU1hE]/‘Zaman PN harga barang naik mencanak!’ - Rafizi [OjJ6hCKU1hE]_0.mp3',
 'alignment': 'dialects_processed_alignment/‘Zaman PN harga barang naik mencanak!’ - Rafizi [OjJ6hCKU1hE]/‘Zaman PN harga barang naik mencanak!’ - Rafizi [OjJ6hCKU1hE]_0.alignment'}

In [9]:
data = mp.multiprocessing(files, loop, cores = 20)
len(data)

100%|███████████████████████████████████████████████████████████████████████████████████| 7359/7359 [01:19<00:00, 92.69it/s]


1131225

In [10]:
d_ = data[1]

In [11]:
d_

{'audio': 'dialects_processed/KILAUAN EMAS ZON UTARA  KEPUTUSAN [tVjxnbDuEfE]/KILAUAN EMAS ZON UTARA  KEPUTUSAN [tVjxnbDuEfE]_1.mp3',
 'alignment': 'dialects_processed_alignment/KILAUAN EMAS ZON UTARA  KEPUTUSAN [tVjxnbDuEfE]/KILAUAN EMAS ZON UTARA  KEPUTUSAN [tVjxnbDuEfE]_1.alignment'}

In [15]:
def loop(rows):
    rows, _ = rows
    result = []
    for r in tqdm(rows):
        f = r['audio']
        
        with open(r['alignment']) as fopen:
            align = json.load(fopen)
            
        c = align
        ts = []
        for c_ in align:
            start = min(timestamps, key=lambda t: abs(t - (c_['start'])))
            end = min(timestamps, key=lambda t: abs(t - (c_['end'])))
            w = c_['text']
            t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
            ts.append(t)

        ts = ''.join(ts)
        new_text = text = f"<|startoftranscript|><|ms|><|transcribeprecise|>{ts}<|endoftext|>"
        
        result.append({
            'new_text': new_text,
            'audio_filename': f
        })
    return result

In [16]:
r = loop((data[:100], 0))

100%|████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 450.60it/s]


In [17]:
r[0]

{'new_text': '<|startoftranscript|><|ms|><|transcribeprecise|><|0.04|> Abang<|0.24|><|0.28|> yakin<|0.66|><|0.72|> tak,<|0.88|><|1.14|> untuk<|1.58|><|1.90|> mewakili<|2.42|><|2.56|> Zon<|2.68|><|2.74|> Utara?<|3.16|><|endoftext|>',
 'audio_filename': 'dialects_processed/KILAUAN EMAS ZON UTARA  KEPUTUSAN [tVjxnbDuEfE]/KILAUAN EMAS ZON UTARA  KEPUTUSAN [tVjxnbDuEfE]_0.mp3'}

In [18]:
r[-1]

{'new_text': '<|startoftranscript|><|ms|><|transcribeprecise|><|0.02|> Kita<|0.34|><|0.78|> yang<|0.88|><|0.92|> transfer<|1.28|><|1.34|> tu,<|1.40|><|1.74|> tajuknya<|2.22|><|2.32|> mesti<|2.54|><|2.66|> bahagian<|3.04|><|3.08|> nombor<|3.26|><|3.30|> satu<|3.56|><|3.64|> ni.<|3.70|><|3.88|> Ni<|3.94|><|4.00|> paling<|4.16|><|4.20|> penting.<|4.48|><|5.24|> Kalau<|5.50|><|5.58|> tajuk<|6.02|><|6.10|> tu<|6.14|><|6.26|> duduk<|6.48|><|6.52|> bahagian<|6.80|><|6.86|> nombor<|7.04|><|7.08|> dua,<|7.24|><|8.46|> kata<|8.80|><|8.98|> studio<|9.20|><|9.28|> tak<|9.36|><|9.40|> boleh<|9.54|><|9.58|> baca.<|9.96|><|endoftext|>',
 'audio_filename': 'dialects_processed/🔴[LIVE] ASAS DATA STUDIO ｜ SIRI 1 [rpM7zMlbwEg]/🔴[LIVE] ASAS DATA STUDIO ｜ SIRI 1 [rpM7zMlbwEg]_139.mp3'}

In [19]:
r = mp.multiprocessing(data, loop, cores = 15)

100%|████████████████████████████████████████████████████████████████████████████████| 75415/75415 [05:52<00:00, 213.90it/s]


In [20]:
import pandas as pd

pd.DataFrame(r).to_parquet('gather-dialects-word.parquet')

In [21]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="gather-dialects-word.parquet",
    path_in_repo="data/dialects_word-00000-of-00001.parquet",
    repo_id="malaysia-ai/STT-Whisper",
    repo_type="dataset",
)

gather-dialects-word.parquet:   0%|          | 0.00/286M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/STT-Whisper/commit/de5a8a33365a59b1711325014080a6b7a51610d3', commit_message='Upload data/dialects_word-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='de5a8a33365a59b1711325014080a6b7a51610d3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/malaysia-ai/STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='malaysia-ai/STT-Whisper'), pr_revision=None, pr_num=None)